In [1]:
import torch
from torch import nn
from d2l import torch as d2l

### 实现丢弃函数
其中 `dropout` 就是丢弃概率 `p` (以 p 概率丢弃神经元)

In [11]:
def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1 # 保证概率值的问题
    if dropout == 1:
        return torch.zeros_like(X) # 神经元置零
    if dropout == 0:
        return X # 保持原输出
    mask = (torch.rand(X.shape) > dropout).float() # 生成一个和 X 形状相同的随机矩阵，大于 p 的为 1 ，其余为 0，最终 mask 由 0 和 1组成
    return mask * X / (1.0 - dropout) # mask 乘以 X，达到去除神经元的作用

测试一下丢弃函数

In [16]:
X = torch.arange(16, dtype=torch.float32).reshape(2, 8)
a = dropout_layer(X, 0.)
b = dropout_layer(X, 0.5)
c = dropout_layer(X, 1.0)

### 定义感知机

使用 Fashion-MNIST 数据集，定义有两个隐藏层的感知机

In [14]:
num_inputs, num_outputs, num_hidden1, num_hidden2 = 784, 10, 256, 256

In [ ]:
dropout1, dropout2 = 0.2, 0.5

class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs, num_hidden1, num_hidden2, is_training = True):
        super(Net, self).__init__()
        self.num_inputs = num_inputs
        self.training = is_training
        self.lin1 = nn.Linear(num_inputs, num_hidden1)
        self.lin2 = nn.Linear(num_hidden1, num_hidden2)
        self.lin3 = nn.Linear(num_hidden2, num_outputs)
        self.relu = nn.ReLU()

    def forward(self, X):
        H1 = self.relu(self.lin1(X.reshape(-1, self.num_inputs)))
        if self.training == True:
            H1 = dropout_layer(H1, dropout1)
        H2 = self.relu(self.lin2(H1))
        if self.training == True:
            H2 = dropout_layer(H2, dropout2)
        out = self.lin3(H2)
        return out